In [15]:
from pgmpy.models import DiscreteBayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.factors.discrete import TabularCPD


In [16]:
model = DiscreteBayesianNetwork([
    ("Conditions","Age"),("Conditions" ,"SmokingStatus"),("Conditions" , "ExerciseLevel"),("Conditions", "FamilyHistory") ]
)

In [17]:
cpd_cond = TabularCPD("Conditions",3,[[0.2],[0.5],[0.3]],state_names={"Conditions": ["HeartDisease","HyperTension","NoDisease"]})

In [18]:
cpd_age = TabularCPD("Age",2,
    [[0.8,0.6,0.3],[0.2,0.4,0.7]],
    evidence=["Conditions"],
    evidence_card=[3],
    state_names={
        "Age" : ["Old","Young"],
        "Conditions": ["HeartDisease","HyperTension","NoDisease"]
    }
)

cpd_smoke = TabularCPD("SmokingStatus",2,
    [[0.75,0.5,0.3],[0.25,0.5,0.7]],
    evidence=["Conditions"],
    evidence_card=[3],
    state_names={
        "SmokingStatus" : ["Yes","No"],
        "Conditions": ["HeartDisease","HyperTension","NoDisease"]
    }
)

cpd_exe = TabularCPD("ExerciseLevel",2,
    [[0.8,0.7,0.6],[0.2,0.3,0.4]],
    evidence=["Conditions"],
    evidence_card=[3],
    state_names={
        "ExerciseLevel" : ["Low","High"],
        "Conditions": ["HeartDisease","HyperTension","NoDisease"]
    }
)

cpd_fam = TabularCPD("FamilyHistory",2,
    [[0.9,0.7,0.4],[0.1,0.3,0.6]],
    evidence=["Conditions"],
    evidence_card=[3],
    state_names={
        "FamilyHistory" : ["Yes","No"],
        "Conditions": ["HeartDisease","HyperTension","NoDisease"]
    }
)

In [19]:
model.add_cpds(cpd_age,cpd_cond,cpd_exe,cpd_fam,cpd_smoke)
assert model.check_model()

In [20]:
inference = VariableElimination(model)

In [21]:
t1 = inference.query(variables=["Conditions"],evidence={"Age": "Old","SmokingStatus":"Yes","ExerciseLevel":"Low","FamilyHistory":"Yes"})
print(t1)

+--------------------------+-------------------+
| Conditions               |   phi(Conditions) |
+==========================+===================+
| Conditions(HeartDisease) |            0.5193 |
+--------------------------+-------------------+
| Conditions(HyperTension) |            0.4418 |
+--------------------------+-------------------+
| Conditions(NoDisease)    |            0.0389 |
+--------------------------+-------------------+


In [22]:

# Task 2: Add BloodPressure node and perform updated inference
cpd_bp = TabularCPD("BloodPressure", 2,
    [[0.85, 0.6, 0.3], [0.15, 0.4, 0.7]],
    evidence=["Conditions"],
    evidence_card=[3],
    state_names={
        "BloodPressure": ["High", "Normal"],
        "Conditions": ["HeartDisease", "HyperTension", "NoDisease"]
    })

model.add_node("BloodPressure")
model.add_edge("Conditions", "BloodPressure")
model.add_cpds(cpd_bp)
assert model.check_model()

t2 = inference.query(
    variables=["Conditions"],
    evidence={
        "Age": "Old",
        "SmokingStatus": "Yes",
        "ExerciseLevel": "Low",
        "FamilyHistory": "Yes",
        "BloodPressure": "High"
    }
)
print("Task 2 Result:")
print(t2)


Task 2 Result:
+--------------------------+-------------------+
| Conditions               |   phi(Conditions) |
+==========================+===================+
| Conditions(HeartDisease) |            0.6146 |
+--------------------------+-------------------+
| Conditions(HyperTension) |            0.3691 |
+--------------------------+-------------------+
| Conditions(NoDisease)    |            0.0163 |
+--------------------------+-------------------+


In [23]:

# Task 3: Compute P(ExerciseLevel = Low | Conditions = HeartDisease)
t3 = inference.query(
    variables=["ExerciseLevel"],
    evidence={"Conditions": "HeartDisease"}
)
print("Task 3 Result:")
print(t3)


Task 3 Result:
+---------------------+----------------------+
| ExerciseLevel       |   phi(ExerciseLevel) |
+=====================+======================+
| ExerciseLevel(Low)  |               0.8000 |
+---------------------+----------------------+
| ExerciseLevel(High) |               0.2000 |
+---------------------+----------------------+
